In [21]:
import pandas as pd
from tabulate import tabulate
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
from math import log

from finviz.screener import Screener
import finviz



In [22]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

/home/shyambv/pythonenv/new_packages/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
ark_df = pd.read_csv('CSV/Ark_Website.csv')

In [24]:
ark_df.head()

,Stock_Ticker,Stock_name,Location,Weight,Website
0,TSLA,TESLA INC,ARKQ,9.71,Ark_Website
1,JD,JD.COM INC-ADR,ARKQ,6.58,Ark_Website
2,TRMB,TRIMBLE INC,ARKQ,5.84,Ark_Website
3,KTOS,KRATOS DEFENSE & SECURITY,ARKQ,5.63,Ark_Website
4,GOOG,ALPHABET INC-CL C,ARKQ,4.76,Ark_Website


In [25]:
ark_df_summary = ark_df.groupby(['Stock_Ticker', 'Stock_name']).sum().reset_index()

In [26]:
ark_df_summary.head()

,Stock_Ticker,Stock_name,Weight
0,1833,PING AN HEALTHCARE AND TECHN,1.22
1,2618,JD LOGISTICS INC,3.64
2,3402,TORAY INDUSTRIES INC,0.14
3,3690,MEITUAN-CLASS B,5.22
4,4477,BASE INC,1.11


In [27]:
overall_ark_funds = ark_df_summary['Stock_Ticker'].to_list()


In [28]:
#Need to execute for all the tickers first time
stock_list = Screener(tickers=overall_ark_funds) 

100%|██████████| 11/11 [00:03<00:00,  3.17it/s]


In [29]:
#Store it for later usage
stock_list.to_csv("CSV/stock_ark_perf.csv")
stock_ark_perf_df = pd.read_csv("CSV/stock_ark_perf.csv")
stock_ark_perf_df.head()

,No.,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
0,1,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General,USA,196.65B,39.62,112.21,0.00%,0
1,2,ACCD,"Accolade, Inc.",Healthcare,Health Information Services,USA,2.97B,-,50.08,0.00%,0
2,3,ACIC,Atlas Crest Investment Corp.,Financial,Shell Companies,USA,618.14M,-,9.93,0.00%,0
3,4,ADPT,Adaptive Biotechnologies Corporation,Healthcare,Biotechnology,USA,5.00B,-,35.61,0.00%,0
4,5,ADSK,"Autodesk, Inc.",Technology,Software - Application,USA,60.57B,46.99,274.47,0.00%,0


In [30]:
stock_ark_perf_df.rename(columns={"Ticker":'Stock_Ticker'}, inplace= True)

#Rename if different field
#ark_df_summary.rename(columns={"Stock Ticker":'Stock_Ticker'}, inplace= True)

In [31]:
stock_ark_perf_df = stock_ark_perf_df.merge(ark_df_summary, on='Stock_Ticker')

In [32]:
stock_ark_perf_df.head()

,No.,Stock_Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume,Stock_name,Weight
0,1,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General,USA,196.65B,39.62,112.21,0.00%,0,ABBVIE INC,0.73
1,2,ACCD,"Accolade, Inc.",Healthcare,Health Information Services,USA,2.97B,-,50.08,0.00%,0,ACCOLADE INC,3.24
2,3,ACIC,Atlas Crest Investment Corp.,Financial,Shell Companies,USA,618.14M,-,9.93,0.00%,0,ATLAS CREST INVESTMENT COR-A,1.66
3,4,ADPT,Adaptive Biotechnologies Corporation,Healthcare,Biotechnology,USA,5.00B,-,35.61,0.00%,0,ADAPTIVE BIOTECHNOLOGIES,1.86
4,5,ADSK,"Autodesk, Inc.",Technology,Software - Application,USA,60.57B,46.99,274.47,0.00%,0,AUTODESK INC,4.16


In [33]:
# Peform some cleaning on market cap. Remove text
stock_ark_perf_df['Market_Cap_log'] = stock_ark_perf_df['Market Cap'].apply(lambda x: x.split("B")[0].split("M")[0])

stock_ark_perf_df['Market_Cap_log'] = stock_ark_perf_df['Market_Cap_log'].apply(lambda x: 10 if x is '-' else float(x))

# Perform log scale
stock_ark_perf_df['Market_Cap_log'] = stock_ark_perf_df['Market_Cap_log'].apply(lambda x: log(x))

# If the market cap is so small, need to add a minor size
stock_ark_perf_df['Market_Cap_log'] = stock_ark_perf_df['Market_Cap_log'].apply(lambda x: 1 if x <1 else x)

# Ark Daily trades

In [34]:
ark_daily = pd.read_pickle('data_pickle/ark_daily_trades')

In [35]:
ark_daily.head()

,Fund,Date,Direction,Ticker,CUSIP,Company,Shares,% of ETF
1,ARKF,06/03/2021,Buy,GOOG,02079K107,ALPHABET INC,4496,0.3008
2,ARKF,06/03/2021,Buy,KSPILI,48581R205,KASPI.KZ JSC,4866,0.0130
3,ARKF,06/03/2021,Sell,TCSLI,87238U203,TCS GROUP HOLDING PLC,72956,0.1433
4,ARKG,06/03/2021,Buy,SGFY,82671G100,SIGNIFY HEALTH INC,32322,0.0099
5,ARKG,06/03/2021,Buy,RPTX,760273102,REPARE THERAPEUTICS INC,13463,0.0053


In [36]:
ark_df_daily_summary = ark_daily.groupby(['Ticker', 'Company','Direction']).sum().reset_index()

In [37]:
ark_df_daily_summary.rename(columns={"Ticker":'Stock_Ticker'}, inplace= True)

In [38]:
ark_df_daily_summary.head()

,Stock_Ticker,Company,Direction,Shares,% of ETF
0,CRSP,CRISPR THERAPEUTICS AG,Buy,8000,0.0044
1,DRNA,DICERNA PHARMACEUTICALS INC,Buy,3406,0.0013
2,GH,GUARDANT HEALTH INC,Sell,12229,0.0178
3,GLEO,GALILEO ACQUISITION CORP,Sell,2900,0.0010
4,GOOG,ALPHABET INC,Buy,4496,0.3008


In [39]:
stock_ark_perf_daily_df = stock_ark_perf_df.merge(ark_df_daily_summary, on='Stock_Ticker')

In [40]:
# Peform some cleaning on market cap. Remove text

stock_ark_perf_daily_df['Market_Cap_log'] = stock_ark_perf_daily_df['Market Cap'].apply(lambda x: x.split("B")[0].split("M")[0])

stock_ark_perf_daily_df['Market_Cap_log'] = stock_ark_perf_daily_df['Market_Cap_log'].apply(lambda x: 10 if x is '-' else float(x))

# Perform log scale

stock_ark_perf_daily_df['Market_Cap_log'] = stock_ark_perf_daily_df['Market_Cap_log'].apply(lambda x: log(x))

In [41]:
stock_ark_perf_daily_df.head()

,No.,Stock_Ticker,Company_x,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume,Stock_name,Weight,Market_Cap_log,Company_y,Direction,Shares,% of ETF
0,44,CRSP,CRISPR Therapeutics AG,Healthcare,Biotechnology,Switzerland,8.34B,-,109.83,0.00%,0,CRISPR THERAPEUTICS AG,5.04,2.121063,CRISPR THERAPEUTICS AG,Buy,8000,0.0044
1,55,DRNA,"Dicerna Pharmaceuticals, Inc.",Healthcare,Biotechnology,USA,2.26B,-,30.47,0.00%,0,DICERNA PHARMACEUTICALS INC,0.12,0.815365,DICERNA PHARMACEUTICALS INC,Buy,3406,0.0013
2,71,GH,"Guardant Health, Inc.",Healthcare,Diagnostics & Research,USA,11.94B,-,117.90,0.00%,0,GUARDANT HEALTH INC,0.19,2.479894,GUARDANT HEALTH INC,Sell,12229,0.0178
3,74,GOOG,Alphabet Inc.,Communication Services,Internet Content & Information,USA,1650.68B,32.04,2404.61,0.00%,0,ALPHABET INC-CL C,7.84,7.408943,ALPHABET INC,Buy,4496,0.3008
4,97,KTOS,"Kratos Defense & Security Solutions, Inc.",Industrials,Aerospace & Defense,USA,3.18B,39.67,25.63,0.00%,0,KRATOS DEFENSE & SECURITY,11.56,1.156881,KRATOS DEFENSE & SECURITY SOLUTIONS INC,Buy,44867,0.0391


In [42]:
from pyvis.network import Network
import pandas as pd

ark_net = Network(height='1500px', width='100%', bgcolor='#222222', font_color='white')


sources = stock_ark_perf_df['Stock_Ticker']
targets = stock_ark_perf_df['Industry']
weights = stock_ark_perf_df['Weight']
market_cap = stock_ark_perf_df['Market Cap']
market_cap_log = stock_ark_perf_df['Market_Cap_log']

title   = stock_ark_perf_df['Company']

edge_data = zip(sources, targets, weights, market_cap, market_cap_log, title)

# Stocks

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    m = e[3]
    m_log = e[4]
    t = e[5]
    
    ark_net.add_node(src, src, size =m_log, title = (str(t), m))
    ark_net.add_node(dst, dst, title=dst, color='#00ff1e')
    ark_net.add_edge(src, dst, value=w, title = str("Overall Holdings: "+str(w)+"%"))



sector = stock_ark_perf_df['Sector']
targets = stock_ark_perf_df['Industry']
weights = stock_ark_perf_df['Weight']

#Industry
edge_data = zip(sector, targets, weights)


for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    
    ark_net.add_node(src, src, title=src, color='#fea200', size =50)
    ark_net.add_node(dst, dst, title=dst)
    ark_net.add_edge(src, dst)
    
    
# Daily Trades


sources = stock_ark_perf_daily_df['Stock_Ticker']
targets = stock_ark_perf_daily_df['Industry']
weights = stock_ark_perf_daily_df['% of ETF']
market_cap = stock_ark_perf_daily_df['Market Cap']
market_cap_log = stock_ark_perf_daily_df['Market_Cap_log']
title   = stock_ark_perf_daily_df['Company_x']
direction = stock_ark_perf_daily_df['Direction']


edge_data = zip(sources, targets, weights, market_cap, market_cap_log, title, direction)

# Stocks

for e in edge_data:
    src = str(" "+e[0])
    dst = e[1]
    w = e[2]
    m = e[3]
    m_log = e[4]
    t = e[5]
    d = e[6]
    
    if d == 'Buy':

        ark_net.add_node(src, src, size =m_log, title = (str(t), m))
        ark_net.add_node(dst, dst, title=dst, color='#00ff1e')
        ark_net.add_edge(src, dst, value=w, title = str("Bought: "+str(w)+"%"), color = "#439c11")
        
    elif d =='Sell':
        
        ark_net.add_node(src, src, size =m_log, title = (str(t), m))
        ark_net.add_node(dst, dst, title=dst, color='#00ff1e')
        ark_net.add_edge(src, dst, value=w, title = str("Sold: "+str(w)+"%"), color ='Red')
        
ark_net.show('ark_net.html')